# Python で気軽に化学・化学工学
# 第 11 章 目標達成に向けて実験条件・製造条件を提案する

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

## 実験計画法

In [ ]:
# 使用するライブラリのインポート
import numpy as np
import pandas as pd
from numpy import matlib

In [ ]:
all_experiments = pd.read_csv('all_experiments.csv', index_col=0, header=0) # 全候補のデータセットの読み込み

In [ ]:
all_experiments # 念のため確認

実験条件の候補のインデックスの作成

In [ ]:
all_experiment_indexes = []
for i in range(all_experiments.shape[0]):
    all_experiment_indexes.append(i)

In [ ]:
all_experiment_indexes # 念のため確認

In [ ]:
number_of_first_experiments = 30 # 最初に実験する候補の数。とりあえず 30 としています

D 最適基準が大きくなるように、実験条件の候補を選択します。

In [ ]:
number_of_random_searches = 1000 # ランダムに実験候補を選択して D 最適基準を計算する繰り返し回数
np.random.seed(111) # 乱数を生成するためのシードを固定
for random_search_number in range(number_of_random_searches):
    # 1. ランダムに候補を選択
    new_selected_experiment_indexes = np.random.choice(all_experiment_indexes, number_of_first_experiments)
    new_selected_experiments = all_experiments.iloc[new_selected_experiment_indexes, :]
    # 2. オートスケーリングした後に D 最適基準を計算
    autoscaled_new_selected_experiments = (new_selected_experiments - new_selected_experiments.mean()) / new_selected_experiments.std()
    xt_x = np.dot(autoscaled_new_selected_experiments.T, autoscaled_new_selected_experiments)
    d_optimal_value = np.linalg.det(xt_x) 
    # 3. D 最適基準が前回までの最大値を上回ったら、選択された候補を更新
    if random_search_number == 0:
        best_d_optimal_value = d_optimal_value.copy()
        selected_experiment_indexes = new_selected_experiment_indexes.copy()
    else:
        if best_d_optimal_value < d_optimal_value:
            best_d_optimal_value = d_optimal_value.copy()
            selected_experiment_indexes = new_selected_experiment_indexes.copy()
selected_experiment_indexes = list(selected_experiment_indexes) # 最後にリスト型に変換

In [ ]:
selected_experiment_indexes # 選択された実験番号の確認

In [ ]:
selected_experiments = all_experiments.iloc[selected_experiment_indexes, :] # 選択された実験条件の候補を確認
selected_experiments

In [ ]:
selected_experiments.corr() # 相関行列の確認

相関係数が小さいことを確認しましょう。特徴量間の情報の重複が小さいように、つまり多重共線性が小さいように候補が選ばれています。

In [ ]:
# 選択された実験条件の候補を csv ファイルに保存
selected_experiments.to_csv('selected_experiments.csv') # 同じ名前のファイルがあるときは上書きされますので注意してください

## 応答曲面法

実験 (今回は数式による y の計算)。実際に実験したときには、実験結果を格納した csv ファイルを `selected_experiments.csv` のインデックスを活用して準備し、そのファイルから y を読み込むのがよいでしょう。

In [ ]:
y_product = 1
for x_number in range(selected_experiments.shape[1]):
    y_product *= np.cos(selected_experiments.iloc[:, x_number] / (x_number + 1) ** 0.5)
y = (1 - 1 / 4000 * (selected_experiments ** 2).sum(axis=1) + y_product) / 2

In [ ]:
y

In [ ]:
y.max() # y の最大値

### OLS による回帰モデルの構築

In [ ]:
# 必要なライブラリのインポート
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import model_selection
import matplotlib.figure as figure
import matplotlib.pyplot as plt

すべてをトレーニングデータとして OLS により回帰モデルを構築します。

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x = (selected_experiments - selected_experiments.mean()) / selected_experiments.std()
autoscaled_y = (y - y.mean()) / y.std()

OLS

In [ ]:
model = LinearRegression() # モデルの宣言

In [ ]:
model.fit(autoscaled_x, autoscaled_y) # 回帰モデルの構築

標準回帰係数

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # Pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients # 念のため確認

In [ ]:
standard_regression_coefficients.to_csv('standard_regression_coefficients.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の値の推定

In [ ]:
estimated_y = pd.DataFrame(model.predict(autoscaled_x)) # y の値を推定し、Pandas の DataFrame 型に変換

In [ ]:
estimated_y = estimated_y * y.std() + y.mean() # スケールをもとに戻します

In [ ]:
estimated_y.index = y.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y # 念のため確認

In [ ]:
estimated_y.to_csv('estimated_y.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y, estimated_y.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y.max(), estimated_y.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.min(), estimated_y.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y, estimated_y) # r2

In [ ]:
metrics.mean_absolute_error(y, estimated_y) # MAE

r<sup>2</sup> の値が 0 に近く、y のばらつきをモデルで説明できていないことがわかります。ただ一応、クロスバリデーションでのモデルの推定性能の検証もします。

In [ ]:
fold_number = 10 # クロスバリデーションの fold 数

In [ ]:
estimated_y_in_cv = model_selection.cross_val_predict(model, autoscaled_x, autoscaled_y, cv=fold_number) # クロスバリデーションで y の値を推定

In [ ]:
estimated_y_in_cv = pd.DataFrame(estimated_y_in_cv) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_in_cv = estimated_y_in_cv * y.std() + y.mean() # スケールをもとに戻します

In [ ]:
estimated_y_in_cv.index = y.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_in_cv.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_in_cv # 念のため確認

In [ ]:
estimated_y_in_cv.to_csv('estimated_y_in_cv.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y, estimated_y_in_cv.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y.max(), estimated_y_in_cv.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.min(), estimated_y_in_cv.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

In [ ]:
metrics.r2_score(y, estimated_y_in_cv) # r2

In [ ]:
metrics.mean_absolute_error(y, estimated_y_in_cv) # MAE

r<sup>2</sup> の値は小さく、モデルで y のばらつきを説明できていないことが分かります。モデルの推定性能は低いといえます。

### 説明変数と目的変数との間の非線形性を考慮した OLS による回帰モデルの構築

以下の 3 つを含む `nonlinear_x` という変数を作成して、説明変数のデータセットとします。2. 3. で目的変数との間の非線形性を表現できることを期待しています。
1. 元の実験条件
2. 各実験条件の二乗項
3. 2 つの実験条件の全組み合わせにおける交差項

In [ ]:
# 1.
nonlinear_x = selected_experiments.copy() 

In [ ]:
# 2. 3. を追加
x_square = selected_experiments ** 2  # 二乗項
for i in range(selected_experiments.shape[1]):
    print(i + 1, '/', selected_experiments.shape[1])
    for j in range(selected_experiments.shape[1]):
        if i == j:  # 二乗項追加
            nonlinear_x = pd.concat([nonlinear_x, x_square.rename(columns={x_square.columns[i]: '{0}^2'.format(x_square.columns[i])}).iloc[:, i]], axis=1)
        elif i < j:  # 交差項追加
            x_cross = selected_experiments.iloc[:, i] * selected_experiments.iloc[:, j]
            x_cross.name = '{0}*{1}'.format(selected_experiments.columns[i], selected_experiments.columns[j])
            nonlinear_x = pd.concat([nonlinear_x, x_cross], axis=1)

In [ ]:
nonlinear_x # 念のため確認

特徴量の標準化(オートスケーリング)

In [ ]:
autoscaled_nonlinear_x = (nonlinear_x - nonlinear_x.mean(axis=0)) / nonlinear_x.std(axis=0, ddof=1)
autoscaled_y = (y - y.mean()) / y.std(ddof=1)

OLS

In [ ]:
model = LinearRegression() # モデルの宣言

In [ ]:
model.fit(autoscaled_nonlinear_x, autoscaled_y) # 回帰モデルの構築

標準回帰係数

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # Pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients # 念のため確認

In [ ]:
standard_regression_coefficients.to_csv('standard_regression_coefficients.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の値の推定

In [ ]:
estimated_y = pd.DataFrame(model.predict(autoscaled_nonlinear_x)) # y の値を推定し、Pandas の DataFrame 型に変換

In [ ]:
estimated_y = estimated_y * y.std() + y.mean() # スケールをもとに戻します

In [ ]:
estimated_y.index = y.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y # 念のため確認

In [ ]:
estimated_y.to_csv('estimated_y.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y, estimated_y.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y.max(), estimated_y.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.min(), estimated_y.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y, estimated_y) # r2

In [ ]:
metrics.mean_absolute_error(y, estimated_y) # MAE

非線形性を考慮する前と比較して、r<sup>2</sup> の値が向上しており、より y のばらつきをモデルで説明できていることがわかります。クロスバリデーションでもモデルの推定性能を検証します。

In [ ]:
fold_number = 10 # クロスバリデーションの fold 数

In [ ]:
estimated_y_in_cv = model_selection.cross_val_predict(model, autoscaled_nonlinear_x, autoscaled_y, cv=fold_number) # クロスバリデーションで y の値を推定

In [ ]:
estimated_y_in_cv = pd.DataFrame(estimated_y_in_cv) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_in_cv = estimated_y_in_cv * y.std() + y.mean() # スケールをもとに戻します

In [ ]:
estimated_y_in_cv.index = y.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_in_cv.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_in_cv # 念のため確認

In [ ]:
estimated_y_in_cv.to_csv('estimated_y_in_cv.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y, estimated_y_in_cv.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y.max(), estimated_y_in_cv.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.min(), estimated_y_in_cv.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

In [ ]:
metrics.r2_score(y, estimated_y_in_cv) # r2

In [ ]:
metrics.mean_absolute_error(y, estimated_y_in_cv) # MAE

クロスバリデーションの結果を見ても、線形のモデルと比較して、r<sup>2</sup> が高く、MAE が小さいことが分かります。非線形モデルのほうが線形モデルより良好な推定性能です。今後は非線形モデルを用いることにします。

### 非線形モデルを用いた、次の実験候補の選択

In [ ]:
# すべての実験条件候補のインデックスから、すでに選択された候補のインデックスを削除し、まだ実験していない候補のインデックスとします
remaining_experiment_indexes = np.delete(all_experiment_indexes, selected_experiment_indexes)

In [ ]:
# まだ実験していない候補
remaining_experiments = all_experiments.iloc[remaining_experiment_indexes, :]

In [ ]:
remaining_experiments # 念のため確認

候補を非線形モデルに入力するため、`remaining_experiments` に対しても以下の 3 つを準備します。`nonlinear_new_x` という、3 つを含む変数を作成します。
1. 元の実験条件
2. 各実験条件の二乗項
3. 2 つの実験条件の全組み合わせにおける交差項

In [ ]:
# 1.
nonlinear_new_x = remaining_experiments.copy() 

In [ ]:
# 2. 3. を追加
x_square = remaining_experiments ** 2  # 二乗項
for i in range(remaining_experiments.shape[1]):
    print(i + 1, '/', remaining_experiments.shape[1])
    for j in range(remaining_experiments.shape[1]):
        if i == j:  # 二乗項追加
            nonlinear_new_x = pd.concat([nonlinear_new_x, x_square.rename(columns={x_square.columns[i]: '{0}^2'.format(x_square.columns[i])}).iloc[:, i]], axis=1)
        elif i < j:  # 交差項追加
            x_cross = remaining_experiments.iloc[:, i] * remaining_experiments.iloc[:, j]
            x_cross.name = '{0}*{1}'.format(remaining_experiments.columns[i], remaining_experiments.columns[j])
            nonlinear_new_x = pd.concat([nonlinear_new_x, x_cross], axis=1)

In [ ]:
nonlinear_new_x # 念のため確認

In [ ]:
# y を推定したい候補の特徴量の標準化(オートスケーリング)。モデルを構築したデータの平均・標準偏差を用いていることに注意
autoscaled_nonlinear_new_x = (nonlinear_new_x - nonlinear_x.mean(axis=0)) / nonlinear_x.std(axis=0, ddof=1)

In [ ]:
estimated_new_y = pd.DataFrame(model.predict(autoscaled_nonlinear_new_x)) # y の値を推定し、Pandas の DataFrame 型に変換

In [ ]:
estimated_new_y = estimated_new_y * y.std() + y.mean() # スケールをもとに戻します

In [ ]:
estimated_new_y.index = remaining_experiments.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_new_y.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_new_y # 念のため確認

In [ ]:
estimated_new_y.to_csv('estimated_new_y.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

#### データ密度による AD の設定

k-NN によって計算されるデータ密度で AD を設定します。トレーニングデータにおける最も近い k 個のサンプルとの距離の平均が小さければ、AD 内と考えられます。

In [ ]:
from sklearn.neighbors import NearestNeighbors # k-NN

In [ ]:
k_in_knn = 3 # k-NN の k

In [ ]:
ad_model = NearestNeighbors(n_neighbors=k_in_knn, metric='euclidean') # k-NN モデルを表す変数の作成

`metric` の種類 (第五回のクラスタリングのときの `metric` とは異なることに注意してください)
- 'euclidean' : ユークリッド距離
- 'manhattan' : マンハッタン距離(シティブロック距離)

など。その他の距離については [こちら](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html) をご覧ください 

In [ ]:
ad_model.fit(autoscaled_nonlinear_x) # k-NN では、トレーニングデータの数値データを model_ad に格納することに対応します

次のセルでは、サンプルごとの k 最近傍サンプルとの距離に加えて、k 最近傍サンプルのインデックス番号も一緒に出力されるため、出力用の変数を 2 つにしています。またトレーニングデータでは k 最近傍サンプルの中に自分も含まれ、自分との距離の 0 を除いた距離を考える必要があるため、k_in_knn + 1 個と設定し直しています。

In [ ]:
knn_distance_between_autoscaled_x_train, knn_index_autoscaled_x_train = ad_model.kneighbors(autoscaled_nonlinear_x, n_neighbors=k_in_knn+1)

In [ ]:
knn_distance_between_autoscaled_x_train # サンプルごとの k最近傍サンプルとの距離

In [ ]:
knn_index_autoscaled_x_train # サンプルごとの k最近傍サンプルのインデックス番号

In [ ]:
knn_distance_between_autoscaled_x_train = pd.DataFrame(knn_distance_between_autoscaled_x_train) # DataFrame型に変換

In [ ]:
knn_distance_between_autoscaled_x_train.index = selected_experiments.index # サンプル名をトレーニングデータのサンプル名にします

In [ ]:
knn_distance_between_autoscaled_x_train # 念のため確認

各サンプルについて、右になるにつれて距離が大きくなることを確認しましょう。すべてのサンプルにおいて、一番左は自分との距離になるため 0.0 になっています。

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train = knn_distance_between_autoscaled_x_train.iloc[:, 1:].mean(axis=1) # 自分以外の k_in_knn 個の距離の平均

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train # 念のため確認

次のセルで距離の平均の小さい順に並び替えます。なお、ascending=False とすると、大きい順に並び替えることができます。

In [ ]:
sorted_mean_of_knn_distance_between_autoscaled_x_train = mean_of_knn_distance_between_autoscaled_x_train.sort_values(ascending=True)

In [ ]:
sorted_mean_of_knn_distance_between_autoscaled_x_train # 念のため確認

In [ ]:
alpha = 0.95

In [ ]:
round(autoscaled_nonlinear_x.shape[0] * alpha) # 距離の小さい α ×100 % のサンプル数

In [ ]:
ad_threshold = sorted_mean_of_knn_distance_between_autoscaled_x_train.iloc[round(autoscaled_nonlinear_x.shape[0] * alpha) - 1] # トレーニング化合物の alpha % が含まれるようにしきい値を設定

In [ ]:
ad_threshold

まだ実験していない候補に対して、AD の内か外かを判定します。

In [ ]:
knn_distance_between_autoscaled_x_train_test, knn_index_autoscaled_x_test = ad_model.kneighbors(autoscaled_nonlinear_new_x)

In [ ]:
knn_distance_between_autoscaled_x_train_test = pd.DataFrame(knn_distance_between_autoscaled_x_train_test) # DataFrame型に変換

In [ ]:
knn_distance_between_autoscaled_x_train_test.index = nonlinear_new_x.index # サンプル名をテストデータのサンプル名にします

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test = knn_distance_between_autoscaled_x_train_test.mean(axis=1) # k_in_knn 個の距離の平均

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test # 念のため確認

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test <= ad_threshold # モデルの適用範囲内のサンプル True

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test > ad_threshold # モデルの適用範囲外のサンプルは True

In [ ]:
estimated_new_y[mean_of_knn_distance_between_autoscaled_x_train_test > ad_threshold] = -10 ** 10 # AD 外の候補においては負に非常に大きい値を代入し、次の候補として選ばれないようにします

In [ ]:
estimated_new_y.to_csv('estimated_new_y_considering_ad.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
estimated_new_y.max() # 推定された y の最大値

In [ ]:
estimated_new_y.idxmax() # 推定された y の最大値の順番

In [ ]:
new_experiments = remaining_experiments.iloc[estimated_new_y.idxmax(), :] #次の実験候補

In [ ]:
new_experiments # 念のため確認

### 実験

今回は実験を想定した数式による y の計算です。実際に実験したときは、y_new のデータを csv ファイルから読み込むか、y_new = ... と直接入力するのがのがよいでしょう。

In [ ]:
y_product = 1
for x_number in range(new_experiments.shape[1]):
    y_product *= np.cos(new_experiments.iloc[:, x_number] / (x_number + 1) ** 0.5)
y_new = (1 - 1 / 4000 * (new_experiments ** 2).sum(axis=1) + y_product) / 2

In [ ]:
y_new # 確認

In [ ]:
y.max() # これまでの最大値を確認

必ずしも最大値が更新されるわけではありません

## 適応的実験計画法

### 実験結果の追加

In [ ]:
estimated_new_y.idxmax()[0] # 新しく実験した候補のインデックス

In [ ]:
selected_experiment_indexes.append(remaining_experiments.index[estimated_new_y.idxmax()[0]]) # これまで選択された実験候補のインデックスに、新しく実験した候補のインデックスを追加

In [ ]:
selected_experiment_indexes # 念のため確認

In [ ]:
y = pd.concat([y, y_new], axis = 0) # y に追加

In [ ]:
y # 念のため確認

In [ ]:
selected_experiments = pd.concat([selected_experiments, new_experiments], axis=0) # 実施済みの実験の候補に追加

In [ ]:
selected_experiments # 念のため確認

In [ ]:
# csv ファイルに保存
selected_experiments.to_csv('selected_experiments.csv') # 同じ名前のファイルがあるときは上書きされますので注意してください

- **説明変数と目的変数との間の非線形性を考慮した OLS による回帰モデルの構築**
- **非線形モデルを用いた、次の実験候補の選択**
- **実験**
- **実験結果の追加**  
  
を繰り返すことで、最適な y を得られる実験候補を探索していきます。これを、**適応的実験計画法 (Adaptive Design of Experiments)** と呼びます

「**説明変数と目的変数との間の非線形性を考慮した OLS による回帰モデルの構築**」のセルからこのセルまでを繰り返し実行すると適応的実験計画法になります。繰り返し実行し、y が大きくなるような候補の探索を行いましょう。